# Evaluación de modelos de texto generativo

Se comenzará la sección configurando el LLM para la generación de texto a partir del código implementado en la sección 3.

![Texto alternativo](./imgs/5.2.png)

La  siguiente  subsección  resume  la  generación  de  texto  que se configuró al final de la sección 3, antes  de  sumergirnos  en  la  evaluación  del  texto  y  el  cálculo  
de  las  pérdidas  de  entrenamiento  y  validación  en  las  subsecciones  posteriores.

## Uso de GPT para generar texto

En esta subsección se configura el LLm y se recapitula brevemente el proceso de generación de texto

In [ ]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)



In [25]:
import torch
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel
from seccion04_ImplementacionGPTGeneracionTexto.gptConfig124M import GPT_CONFIG_124M

#importacion de librerias necesarias y acortamiento del context length de la configuracion de GPT (1024 tokens a  256)
GPT_CONFIG_124M["context_length"] = 256
print(GPT_CONFIG_124M)

{'vocab_size': 50257, 'context_length': 256, 'emb_dim': 768, 'n_heads': 12, 'n_layers': 12, 'drop_rate': 0.1, 'qkv_bias': False}


In [26]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

Esta  modificación  reduce  las  demandas  computacionales  del  entrenamiento  del  modelo,  haciendo  posible  realizar  el  entrenamiento  en  una  computadora  portátil  estándar.

Originalmente,  el  modelo  GPT2,  con  124  millones  de  parámetros,  se  configuró  para  gestionar  hasta  1024  tokens. 

Utilizando  la  instancia  GPTmodel ,  adoptamos  la  función  generate_text_simple  presentada anteriorment e se introducen dos  funciones  útiles:  text_to_token_ids  y  token_ids_to_text.  Estas  funciones  facilitan  la  conversión  entre  representaciones  de  texto  y  tokens.

![Texto alternativo](./imgs/5.3.png)

- Primero,  el  tokenizador  convierte  el  texto  de  entrada  en  una  serie  de  identificadores  de  token.
- Segundo,  el  modelo  recibe  estos  identificadores  de  token  y  genera  los  logits  correspondientes,  que  son  vectores  que  representan  la  distribución  de  probabilidad  de  cada  token  del  vocabulario.
-  Tercero,  estos  logits  se  convierten  de  nuevo  en  identificadores  de  token,  que  el  tokenizador  decodifica  en  texto  legible,  completando  así  el  ciclo  de  entrada  a  salida  textual.

In [29]:
from seccion04_ImplementacionGPTGeneracionTexto.generateTextSimple import generate_text_simple
import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    # 1. squeeze(0): Quita la dimensión del "lote" que añadió unsqueeze.
    #Es como sacar la lista de la caja para poder leerla.
    #Ej: tensor([[25134]]) -> tensor([25134]), forma: [1]
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    # 2. Convierte el tensor 1D a una lista y luego a texto.
    # Ej: tensor([25134]) -> [25134] -> "hola"

    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))



Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


El modelo sigue sin producir texto coherente porque todavia no se ha entrenado.

Para  definir  qué  hace  que  un  texto  sea  "coherente"  o  de  "alta  calidad", se debe implementar un  método  numérico  para  evaluar  el  contenido  generado.  Este  enfoque  nos  permitirá  monitorear  y  mejorar  el  rendimiento  del  modelo  durante  su  proceso  de  entrenamiento.

## Cálculo de la perdida de generación de texto

En esta subsección se exploraran técnicas para evaluar numericamnte la calidad del texto generado durante el entrenamiento mediante el cálculo de la llamada función de pérdidda de generación de texto.

![Texto alternativo](./imgs/5.1.png)

Es necesario realizar todos estos pasos indicados en la figura antes de calcular una pérdida que mida la calidad de l texto generado.

La figura describe  el  proceso  de  generación  de  texto  con  un  vocabulario  reducido  de  7  tokens  para  que  esta  imagen  quepa  en  una  sola  página.  Sin  embargo,  GPTModel  trabaja  con  un  vocabulario  mucho  mayor,  compuesto  por  50 257  palabras;  por  lo  tanto,  los  ID  de  token  en  los  siguientes  códigos  estarán  comprendidos  entre  0  y  50 256,  en  lugar  de  entre  0  y  6.

Definición de dos entradas con sus objetivos:

In [ ]:
#Paso 1
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                        [40, 1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [107,  588, 11311]]) #  " really like chocolate"]

Los  objetivos  son  las  entradas,  pero  desplazados  una  posición  hacia  adelante.
Esta  estrategia  de  desplazamiento  es  crucial  para  enseñar  al  modelo  a  predecir  el  siguiente  token  en  una  secuencia. Cuando  introducimos  las  entradas  en  el  modelo  para  calcular  vectores  logit  para  los  dos  ejemplos  de  entrada,  cada  uno  compuesto  por  tres  tokens,  y  aplicamos  la  función  softmax  para  transformar  estos  valores  logit  en  puntuaciones  de  probabilidad. 

In [ ]:
#Paso 2: logits a scores de probabilidad
with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(probas.shape) 
#Paso 3 y 4: argmax para obtener el correspondiente token ID 
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Ids token: ", token_ids)

torch.Size([2, 3, 50257])
Ids token:  tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


Dado  que  se tiene  2  lotes  de  entrada,  cada  uno  con  3  tokens,  aplicar  la  función  argmax  a  los  puntajes  de  probabilidad  produce  2  conjuntos  de  salidas,  cada  uno  con  3  identificaciones  de  tokens  predichas.

In [ ]:
#Paso 5: Convertir IDs a texto
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}") #verdadera salida
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}") #salida del modelo

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


El  modelo  produce  texto  aleatorio  diferente  del  texto  objetivo  porque  aún  no  se  ha  entrenado. 

Ahora se debe evaluar numéricamente el rendimiento del texto generado por el modelo mediante la llamada pérdida. Esto no es solo útil para medir la calidad del texto generado, sino que también es un elemento fundamental para implementar posteriormente la función de entrenamiento, que se utiliza para actualizar el peso del modelo y mejorar el texto generado.

![Texto alternativo](./imgs/5.5.png)